<a href="https://colab.research.google.com/github/Amitparikh1/CADS_DataCleaningWorkshop/blob/main/CADS_DataCleaningWorkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Packages


In [1]:
import pandas as pd

Read in the Data 

In [2]:
df_master = pd.read_csv("https://raw.githubusercontent.com/Amitparikh1/CADS_DataCleaningWorkshop/main/messy_data.csv")
df_master = df_master.drop(columns="Unnamed: 0")
df = df_master
df.head()

,id,university,grade,gender,exercise_type,exercise_amount
0,0,UNC,Senior,Male,Indoor,2.5
1,0,UNC,Fourth-year,Male,Outdoor,3.0
2,1,UNC,Senior,Other,Indoor,2.5
3,1,UNC,Fourth-year,Other,Outdoor,2.5
4,2,UNC,Junior,Male,Indoor,2.0


In [3]:
### SCENARIO ### 
# Two UNC professors are doing research about how often students exercise indoors and outdoors 
# The professors conduct two separate surveys, one asking about outdoor exercise and the other about indoor exercise
# Now, they've combined their results and want to analyze the results 

Drop Unnecessary Columns

In [4]:
# Let's take a look at how many unique values are in each row 
print("Number of unique vals per column:")
for col in df.columns:
  print(col + ": " + str(len(df[col].unique())))

Number of unique vals per column:
id: 500
university: 1
grade: 8
gender: 3
exercise_type: 2
exercise_amount: 8


In [5]:
# What should we do about rows that only have 1 unique value? 
df = df.drop(columns=['university'])

Change categories

In [6]:
# What are the categories for 'gender'
print(df['gender'].unique())

['Male' 'Other' 'Female']


In [7]:
# Dummy Encoding using Python list comprehension
# Want n - 1 variables where n is the number of categories - this will avoid multi-collinearity
# We want to do this rather than just "0,1,2" in one category because a ranking doesn't make sense between the categories
df['Male'] = [1 if x == 'Male' else 0 for x in df['gender']]
df['Female'] = [1 if x == 'Female' else 0 for x in df['gender']]

# Reorder columns
df = df.drop(columns=['gender'])
df = df[['id', 'grade', 'Male', 'Female', 'exercise_type', 'exercise_amount']]

In [8]:
df.head(10)

,id,grade,Male,Female,exercise_type,exercise_amount
0,0,Senior,1,0,Indoor,2.5
1,0,Fourth-year,1,0,Outdoor,3.0
2,1,Senior,0,0,Indoor,2.5
3,1,Fourth-year,0,0,Outdoor,2.5
4,2,Junior,1,0,Indoor,2.0
5,2,Third-year,1,0,Outdoor,2.0
6,3,Senior,1,0,Indoor,NaN
7,3,Senior,1,0,Outdoor,3.0
8,4,Sophomore,0,0,Indoor,2.5
9,4,Sophomore,0,0,Outdoor,NaN


Fix formatting of 'grade' column

In [9]:
# Ordinal Encoding
# Grade column has inconsistent formatting. Ex. "Senior" and "Fourth-year"
# Grades have a ranking so we can number these from 1-4
def format_grades(grade):
  if grade == "Senior" or grade == "Fourth-year":
    return 4
  elif grade == "Junior" or grade == "Third-year":
    return 3
  elif grade == "Sophomore" or grade == "Second-year":
    return 2
  elif grade == "Freshman" or grade == "First-year":
    return 1

df['grade'] = df['grade'].apply(lambda x: format_grades(x))

In [10]:
df.head()

,id,grade,Male,Female,exercise_type,exercise_amount
0,0,4,1,0,Indoor,2.5
1,0,4,1,0,Outdoor,3.0
2,1,4,0,0,Indoor,2.5
3,1,4,0,0,Outdoor,2.5
4,2,3,1,0,Indoor,2.0


Tidy the data

In [11]:
df_temp = df[['id','exercise_type', 'exercise_amount']]
df_tidy = df_temp.pivot(index="id", columns="exercise_type")
df_tidy = df_tidy['exercise_amount'][['Indoor', 'Outdoor']]
df_tidy.head()

exercise_type,Indoor,Outdoor
id,,
0,2.5,3.0
1,2.5,2.5
2,2.0,2.0
3,NaN,3.0
4,2.5,NaN


In [12]:
df_temp = df.drop(columns=['exercise_type', 'exercise_amount'])
df_tidy = df_tidy.merge(right=df_temp, how='outer', on='id')
df_tidy = df_tidy.drop_duplicates()
df_tidy.head()

,id,Indoor,Outdoor,grade,Male,Female
0,0,2.5,3.0,4,1,0
2,1,2.5,2.5,4,0,0
4,2,2.0,2.0,3,1,0
6,3,NaN,3.0,4,1,0
8,4,2.5,NaN,2,0,0


In [13]:
df_tidy = df_tidy[['id', 'grade', 'Male', 'Female', 'Indoor', 'Outdoor']]
df_tidy.head()

,id,grade,Male,Female,Indoor,Outdoor
0,0,4,1,0,2.5,3.0
2,1,4,0,0,2.5,2.5
4,2,3,1,0,2.0,2.0
6,3,4,1,0,NaN,3.0
8,4,2,0,0,2.5,NaN


Impute missing values in 'Indoor amount' and 'Outdoor amount'

In [14]:
# We have lots of na values in 'Indoor' and 'Outdoor' Exercise amounts 
print(df_tidy.isna().sum())

id           0
grade        0
Male         0
Female       0
Indoor      86
Outdoor    106
dtype: int64


In [15]:
# Simple Method

# Replace with Mean, Median, or Mode
df_med_imputation = df_tidy
indoor_med = df_tidy['Indoor'].median()
df_med_imputation['Indoor'] = df_tidy['Indoor'].fillna(indoor_med)
outdoor_med = df_tidy['Outdoor'].median()
df_med_imputation['Outdoor'] = df_tidy['Outdoor'].fillna(outdoor_med)

print("Indoor Median: " + str(indoor_med))
print("Outdoor Median: " + str(outdoor_med))
df_med_imputation.head()

Indoor Median: 1.5
Outdoor Median: 1.5


,id,grade,Male,Female,Indoor,Outdoor
0,0,4,1,0,2.5,3.0
2,1,4,0,0,2.5,2.5
4,2,3,1,0,2.0,2.0
6,3,4,1,0,1.5,3.0
8,4,2,0,0,2.5,1.5


In [ ]:
# Regression or K-NN Imputation
# Try these on your own! 